In [62]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
import scipy
from scipy.optimize import newton
from scipy.optimize import fixed_point

"""
use beta=0.95, 0.97
lambdah=0.41, 0.45, 0.49
"""


class gsc(object):
    def __init__(self, beta, lambdah):
        self.beta=beta
        self.lambdah=lambdah
        
    def lnwage(self, low):
        beta=self.beta
        temp1=(1-beta)/((1-low)*beta*(low+beta-1))
        temp2=(low+beta-1)/low
        return np.exp(temp1)*temp2

    """
    high=1-low
    llow=low+beta-1
    """
    def stage(self, low):
        beta=self.beta
        lambdah=self.lambdah
        temp1=(low+beta-1)*np.exp(-lambdah)
        temp2=np.sqrt(temp1**2+(1-low)*(1-low+2*(low+beta-1)))
        temp3=(temp1+temp2)/(1-low+2*(low+beta-1))
        return -np.log(temp3)/lambdah

    def wage(self, low):
        lambdah=self.lambdah
        return (np.exp(2*lambdah*self.stage(low))+1)/2


    def dwage(self, low):
        return self.lnwage(low)-self.wage(low)
    
    def nofriction(self):
        beta=self.beta
        lambdah=self.lambdah
        full_low1=float(fsolve(self.dwage, 0.3))
        full_wage1=float(self.wage(full_low1))
        full_stage1=float(self.stage(full_low1))
        full_learn1=float((1-beta)/full_low1)
        print(full_wage1,"\n",
              full_stage1, "\n",
              full_low1, "\n",
              full_learn1, "\n")
        full_low2=float(fsolve(self.dwage, 0.98))
        full_wage2=float(self.wage(full_low2))
        full_stage2=float(self.stage(full_low2))
        full_learn2=float((1-beta)/full_low2)
        print(full_wage2,"\n",
              full_stage2, "\n",
              full_low2, "\n",
              full_learn2, "\n")
        
    def friction(self):
        beta=self.beta
        lambdah=self.lambdah
        def g(w):
            def f(x):
                return self.lnwage(x)-w
            return (fsolve(f, 0.1), fsolve(f, 0.8))
        def glow(w):
            return (g(w)[0]+g(w)[1])/2
        def newwage(w):
            return self.wage(glow(w))
        equi_wage=float(fixed_point(newwage, 1.16))
        equi_lownorth=float(g(equi_wage)[0])
        equi_lowsouth=float(g(equi_wage)[1])
        equi_stage=float(self.stage(low(equi_wage)))
        equi_learnsouth=float((1-beta)/equi_lowsouth)
        equi_learnnorth=float((1-beta)/equi_lownorth)
        print(equi_wage,"\n",
             equi_stage, "\n",
             equi_lownorth, "\n",
              equi_learnnorth, "\n",
             equi_lowsouth, "\n",
             equi_learnsouth)

In [121]:

beta=0.96
lambdah=0.41

def invertstatic(w):
    def t(w):
        return 1/(2*w-1)**0.5
    def f(low):
        return (1-low+2*(low+beta-1))*t(w)**2-2*(low+beta-1)*np.exp(-lambdah)*t(w)
    return fixed_point(f, 0.1)
g1=gsc(0.95, lambdah)
g2=gsc(0.97, lambdah)
def invertdynamic1(w):
    def f(low):
        return g1.lnwage(low)-w
    return fsolve(f, 0.1)
def invertdynamic2(w):
    def f(low):
        return g2.lnwage(low)-w
    return fsolve(f, 0.1)
def dw(w):
    return 2*invertstatic(w)-invertdynamic1(w)-invertdynamic2(w)
wstar=float(fsolve(dw, 1.05))
lowsouth=float(invertdynamic1(wstar))
lownorth=float(invertdynamic2(wstar))
learnsouth=float((1-0.95)/lowsouth)
learnnorth=float((1-0.97)/lownorth)
sstar=float(np.log(2*wstar-1)/(2*lambdah))
print(wstar, "\n",
      sstar, "\n",
      lownorth, "\n",
      learnnorth, "\n",
      lowsouth, "\n",
      learnsouth
     )

1.0597572530305912 
 0.1376769690216081 
 0.302388877607516 
 0.09920999818961054 
 0.3555090255375633 
 0.14064340539426617


C:\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:236: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
C:\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:236: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)


In [17]:
g1=gsc(0.97, 0.49)
g1.nofriction()
g1.friction()

full wagehigh=[ 1.06888374] 
full critical stage=[ 0.13170204] 
full low skill=[ 0.19084316] 
full learning intensity=[ 0.15719715] 

full wagelow=[ 1.71097804] 
full critical stage=[ 0.90262807] 
full low skill=[ 0.94034327] 
full learning intensity=[ 0.03190324] 

global wage=[ 1.21138251] 
global critical stage=[ 0.35979815] 
north low skill=[ 0.08213941] 
north learning intensity=[ 0.36523271] 
south low skill=[ 0.83105444] 
south learning intensity=[ 0.03609872]
